In [10]:
import os,datetime
from PIL import Image,ExifTags
import re
import hashlib
from hachoir.parser import createParser
import warnings
from hachoir.metadata import extractMetadata

In [34]:
def filename_to_date_via_format(file, formatt):
    result = datetime.datetime.strptime(file, formatt).strftime("%Y:%m:%d %H:%M:%S")
    return result

In [68]:
def creation_date_of_video(filename):
    
    # Special cases..
    #video-2012-12-29-18-40-59.mp4
    if re.search('video-[0-9]{4}-[0-9]{2}-[0-9]{2}-[0-9]{2}-[0-9]{2}-[0-9]{2}.mp4', filename) is not None:
        
        return filename_to_date_via_format(re.sub('.+video', 'video', filename), 'video-%Y-%m-%d-%H-%M-%S.mp4')
    
    parser = createParser(filename)
    metadata = extractMetadata(parser)
    return metadata.get('creation_date').strftime("%Y_%m_%d %H_%M_%S")

In [75]:
# Don't call this directly..
def get_exif_date_proxy(file):
    try:
        img = Image.open(file)
        try:
            return img._getexif()[36867]
        except:
            try: 
                return img._getexif()[306]
            except:    
                return None
    except:
         try:
            return creation_date_of_video(file)
         except:
            return None


        
def get_exif_date(file):
    result = get_exif_date_proxy(file)
    if  result is None:
        return None
    else:
        result = re.sub(':', '_', result)
        if re.search('[0-9]{4}_[0-9]{2}_[0-9]{2} [0-9]{2}_[0-9]{2}_[0-9]{2}', result) is None:
            return None
        else:
            dateparsed = datetime.datetime.strptime(result, "%Y_%m_%d %H_%M_%S").year
            if dateparsed < 1991 or dateparsed > 2025:
                print('Tarihten red: ' + file)
                return None
            return result


In [76]:
def rename_preserving_extension(file, name):
    filename = re.sub('^\.\/', '', str(file))
    correctName = name + re.search('\.([^\.]+)$', filename).group()
    correctName = re.sub('[^0-9A-Z-_\.a-z\/]', '', correctName)
    os.rename(file, correctName);

In [77]:
def exif_create_folder_for_date(file):
    result = get_exif_date(file)
    result = re.sub(':', '_', result)
    result = re.sub('.{9}$', '', result)
    year = re.search('[0-9]{4}', result).group()
    try:
        os.mkdir(year)
    except:
        None
    try:
        os.mkdir(year + '/' + result)
    except:
        None
    return year + '/' + result


In [78]:
def exif_get_hour_of_creation(file):
    result = get_exif_date(file)
    result = re.sub(':', '_', result)
    result = re.sub('^.{11}', '', result)
    return result

In [79]:
def md5sum(filename):
    md5_hash = hashlib.md5()
    with open(filename,"rb") as f:
        # Read and update hash in chunks of 4K
        for byte_block in iter(lambda: f.read(4096),b""):
            md5_hash.update(byte_block)
        return re.search('^.{5}', md5_hash.hexdigest()).group()

In [81]:
directory = os.fsencode('.')
    
for root,d_names,f_names in os.walk('.'):
    if re.search('^(\.\/[0-9]{4}|\.\/[^\.]*\.)', str(root)) is None:
        print ("processing path: " + root)
        for f in f_names:
            file = os.path.join(root, f)
        
            if get_exif_date(file) is None:
                None # Skip
            else:
                folder = exif_create_folder_for_date(file)
                newname = folder + '/' + exif_get_hour_of_creation(file) + '_' + md5sum(file)
                rename_preserving_extension(file, newname)
                print(file + " -----> " + newname)
    else:
        print("ignored path: " + root)
print('end')

processing path: .


[warn] [<NdsFile>] Error when getting size of 'header': delete it
[warn] [<NdsFile>] Error when getting size of 'header': delete it
[warn] [<NdsFile>] Error when getting size of 'header': delete it
[warn] [<NdsFile>] Error when getting size of 'header': delete it
[warn] [<NdsFile>] Error when getting size of 'header': delete it
[warn] [<NdsFile>] Error when getting size of 'header': delete it


ignored path: ./2013
ignored path: ./2013/2013_01_01
ignored path: ./2012
ignored path: ./2012/2012_12_31
ignored path: ./2012/2012_12_30
ignored path: ./2012/2012_12_29
processing path: ./util
processing path: ./Pictures
processing path: ./Pictures/data-download-16
processing path: ./Pictures/data-download-11
processing path: ./Pictures/newset
processing path: ./Pictures/data-download-10
processing path: ./Pictures/data-download-10/__pycache__


[warn] Skip parser 'PythonCompiledFile': Unknown magic number (3413)
[warn] [<NdsFile>] Error when getting size of 'header': delete it


processing path: ./Pictures/Rusya 2012
processing path: ./Pictures/data-download-3
processing path: ./Pictures/data-download-4
processing path: ./Pictures/data-download-5
processing path: ./Pictures/data-download-2
processing path: ./Pictures/Earlier
processing path: ./Pictures/Earlier/1980_01_01
Tarihten red: ./Pictures/Earlier/1980_01_01/00_00_21_275ee.jpg
Tarihten red: ./Pictures/Earlier/1980_01_01/00_00_43_e4308.jpg
Tarihten red: ./Pictures/Earlier/1980_01_01/00_00_19_60829.jpg
Tarihten red: ./Pictures/Earlier/1980_01_01/00_00_49_69f6a.jpg
Tarihten red: ./Pictures/Earlier/1980_01_01/00_00_23_b303d.jpg
Tarihten red: ./Pictures/Earlier/1980_01_01/00_06_17_81757.jpg
Tarihten red: ./Pictures/Earlier/1980_01_01/00_00_48_5aff7.jpg
Tarihten red: ./Pictures/Earlier/1980_01_01/00_00_05_f2f6f.jpg
Tarihten red: ./Pictures/Earlier/1980_01_01/00_00_03_7c823.jpg
Tarihten red: ./Pictures/Earlier/1980_01_01/00_00_04_755b8.jpg
Tarihten red: ./Pictures/Earlier/1980_01_01/00_05_14_66959.jpg
Tarihten 

...

Tarihten red: ./Pictures/Later/2099_05_25/20_35_17_55420.jpg
Tarihten red: ./Pictures/Later/2099_05_25/21_34_44_fba0e.JPG
Tarihten red: ./Pictures/Later/2099_05_25/20_41_53_6af91.jpg
Tarihten red: ./Pictures/Later/2099_05_25/20_42_10_47dd9.jpg
Tarihten red: ./Pictures/Later/2099_05_25/20_42_10_fbceb.JPG
Tarihten red: ./Pictures/Later/2099_05_25/20_40_15_3e946.jpg
Tarihten red: ./Pictures/Later/2099_05_25/21_36_45_e7d75.JPG
Tarihten red: ./Pictures/Later/2099_05_25/21_28_08_3bcbb.jpg
Tarihten red: ./Pictures/Later/2099_05_25/20_29_14_db7ff.jpg
Tarihten red: ./Pictures/Later/2099_05_25/20_35_48_a2841.JPG
Tarihten red: ./Pictures/Later/2099_05_25/21_42_31_8a263.jpg
Tarihten red: ./Pictures/Later/2099_05_25/22_20_35_a5c09.jpg
Tarihten red: ./Pictures/Later/2099_05_25/20_41_22_d55c6.JPG
Tarihten red: ./Pictures/Later/2099_05_25/21_26_41_d99d3.JPG
Tarihten red: ./Pictures/Later/2099_05_25/22_21_13_fb7b4.jpg
Tarihten red: ./Pictures/Later/2099_05_25/20_24_56_2e7dc.jpg
Tarihten red: ./Pictures

Tarihten red: ./Pictures/Later/2099_05_25/00_14_57_1603d.JPG
Tarihten red: ./Pictures/Later/2099_05_25/22_28_41_eb74b.jpg
Tarihten red: ./Pictures/Later/2099_05_25/20_05_03_928b2.JPG
Tarihten red: ./Pictures/Later/2099_05_25/22_21_48_1a7d7.JPG
Tarihten red: ./Pictures/Later/2099_05_25/20_29_53_1fd04.JPG
Tarihten red: ./Pictures/Later/2099_05_25/22_24_11_204fa.jpg
Tarihten red: ./Pictures/Later/2099_05_25/21_47_53_c8545.jpg
Tarihten red: ./Pictures/Later/2099_05_25/21_26_41_7ef2b.jpg
Tarihten red: ./Pictures/Later/2099_05_25/20_42_22_861a6.jpg
Tarihten red: ./Pictures/Later/2099_05_25/04_02_33_7d2aa.JPG
Tarihten red: ./Pictures/Later/2099_05_25/21_28_13_5ce81.JPG
Tarihten red: ./Pictures/Later/2099_05_25/20_46_33_e0bf2.JPG
Tarihten red: ./Pictures/Later/2099_05_25/20_24_38_cca61.JPG
Tarihten red: ./Pictures/Later/2099_05_25/20_28_46_7ba24.JPG
Tarihten red: ./Pictures/Later/2099_05_25/20_41_48_ad909.jpg
Tarihten red: ./Pictures/Later/2099_05_25/20_04_53_c0db4.jpg
Tarihten red: ./Pictures

Tarihten red: ./Pictures/Later/2099_05_24/22_23_44_57ad6.jpg
Tarihten red: ./Pictures/Later/2099_05_24/21_08_14_94ec6.JPG
Tarihten red: ./Pictures/Later/2099_05_24/21_34_33_c807e.jpg
Tarihten red: ./Pictures/Later/2099_05_24/22_39_59_369be.jpg
Tarihten red: ./Pictures/Later/2099_05_24/20_29_46_64d65.jpg
Tarihten red: ./Pictures/Later/2099_05_24/20_01_30_2ddcd.JPG
Tarihten red: ./Pictures/Later/2099_05_24/20_43_55_cab45.JPG
Tarihten red: ./Pictures/Later/2099_05_24/21_33_24_25c27.JPG
Tarihten red: ./Pictures/Later/2099_05_24/20_55_54_90aba.jpg
Tarihten red: ./Pictures/Later/2099_05_24/21_33_24_42a5c.jpg
Tarihten red: ./Pictures/Later/2099_05_24/20_03_12_99ae9.JPG
Tarihten red: ./Pictures/Later/2099_05_24/22_40_19_e79cf.JPG
Tarihten red: ./Pictures/Later/2099_05_24/20_02_04_d5c72.JPG
Tarihten red: ./Pictures/Later/2099_05_24/22_09_36_37977.jpg
Tarihten red: ./Pictures/Later/2099_05_24/22_41_49_7e516.jpg
Tarihten red: ./Pictures/Later/2099_05_24/22_39_06_a9c9d.JPG
Tarihten red: ./Pictures

Tarihten red: ./Pictures/Later/2099_05_24/19_58_45_ea48e.jpg
Tarihten red: ./Pictures/Later/2099_05_24/21_09_50_08385.jpg
Tarihten red: ./Pictures/Later/2099_05_24/20_07_52_bdbaf.JPG
Tarihten red: ./Pictures/Later/2099_05_24/20_02_56_33ace.jpg
Tarihten red: ./Pictures/Later/2099_05_24/20_53_25_2ee83.jpg
Tarihten red: ./Pictures/Later/2099_05_24/20_53_35_ed1cc.jpg
Tarihten red: ./Pictures/Later/2099_05_24/21_03_28_129d2.jpg
Tarihten red: ./Pictures/Later/2099_05_24/20_45_09_1bf97.JPG
Tarihten red: ./Pictures/Later/2099_05_24/20_19_38_92ed6.JPG
Tarihten red: ./Pictures/Later/2099_05_24/22_24_29_8dd84.jpg
Tarihten red: ./Pictures/Later/2099_05_24/21_10_09_6d694.jpg
Tarihten red: ./Pictures/Later/2099_05_24/20_17_23_8a8a0.JPG
Tarihten red: ./Pictures/Later/2099_05_24/21_10_34_ed559.JPG
Tarihten red: ./Pictures/Later/2099_05_24/23_14_30_238f6.JPG
Tarihten red: ./Pictures/Later/2099_05_24/20_43_55_f521f.jpg
Tarihten red: ./Pictures/Later/2099_05_24/22_39_59_4ac1e.JPG
Tarihten red: ./Pictures

Tarihten red: ./Pictures/Later/2099_05_26/19_28_49_e97ce.jpg
Tarihten red: ./Pictures/Later/2099_05_26/02_01_14_86ee2.JPG
Tarihten red: ./Pictures/Later/2099_05_26/22_19_18_cffa6.jpg
Tarihten red: ./Pictures/Later/2099_05_26/19_56_29_a66d2.JPG
Tarihten red: ./Pictures/Later/2099_05_26/19_25_03_f66be.JPG
Tarihten red: ./Pictures/Later/2099_05_26/20_22_04_610c4.JPG
Tarihten red: ./Pictures/Later/2099_05_26/04_59_39_4c67d.JPG
Tarihten red: ./Pictures/Later/2099_05_26/20_22_14_e3a83.JPG
Tarihten red: ./Pictures/Later/2099_05_26/19_32_28_13fe5.JPG
Tarihten red: ./Pictures/Later/2099_05_26/01_17_47_b4597.JPG
Tarihten red: ./Pictures/Later/2099_05_26/20_00_38_c8ad5.jpg
Tarihten red: ./Pictures/Later/2099_05_26/20_00_38_a201c.JPG
Tarihten red: ./Pictures/Later/2099_05_26/21_06_57_897cb.jpg
Tarihten red: ./Pictures/Later/2099_05_26/22_18_56_c1a81.jpg
Tarihten red: ./Pictures/Later/2099_05_26/19_24_58_a8a3f.JPG
Tarihten red: ./Pictures/Later/2099_05_26/20_12_23_18793.JPG
Tarihten red: ./Pictures